In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
%conda list

# packages in environment at /home/ec2-user/anaconda3/envs/tensorflow2_p36:
#
# Name                    Version                   Build  Channel
_anaconda_depends         5.1.0                    py36_2  
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
absl-py                   0.14.1                   pypi_0    pypi
aiobotocore               1.3.0                    pypi_0    pypi
aiohttp                   3.7.4.post0              pypi_0    pypi
aioitertools              0.7.1              pyhd8ed1ab_0    conda-forge
alabaster                 0.7.12                     py_0    conda-forge
anaconda-client           1.7.2                      py_0    conda-forge
anaconda-project          0.9.1              pyhd8ed1ab_0    conda-forge
anyio                     2.1.0            py36h5fab9bb_0    conda-forge
appdirs                   1.4.4              pyh9f0ad1d_0    conda-forge
argh         


Note: you may need to restart the kernel to use updated packages.


In [59]:
from iid_modeling_loocv import iid_modeling_loocv

import os
import pandas as pd
import time
# import tensorflow as tf
# print(tf.__version__)
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score,roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils import resample

## Loading parquet files in dictionary

Need to update `specific folder` variable as necessary

In [19]:
loaded_parquet_df = {}
directory_location = '../data/dwt_df_parquet/'
specific_folder = 'seg_0.5_hop_0.5_ovlap_0.5/'
for dirName, _, fileList in os.walk(f'{directory_location}{specific_folder}'):
    for fname in fileList:
        if fname.endswith('parquet'):
            loaded_parquet_df[fname] = pd.read_parquet(f'{directory_location}{specific_folder}{fname}')
            # print(fname)
# parq = pd.read_parquet('../data/dwt_df_parquet/seg_0.5_hop_0.5_ovlap_0.5/2DCEgkjSeRQyMt8KovG8vF.parquet')

In [20]:
total_obs = 0
for key, val in loaded_parquet_df.items():
    print(f'{key}: {val.shape}')
    total_obs += val.shape[0]
    print(f'{key} has {val.isnull().sum().sum()} missing values')

print(f'Total number of observations: {total_obs}')

1jfWVd1sTapjjkQwDM4yQX.parquet: (714, 5514)
1jfWVd1sTapjjkQwDM4yQX.parquet has 0 missing values
0TkGYYIPwRqx8xzP0XGvRG.parquet: (3014, 5514)
0TkGYYIPwRqx8xzP0XGvRG.parquet has 0 missing values
79aiOALBohH9vEIUHPAoRj.parquet: (5603, 5514)
79aiOALBohH9vEIUHPAoRj.parquet has 0 missing values
1XgTQnRlfJ0zpDdg2DccbR.parquet: (5630, 5514)
1XgTQnRlfJ0zpDdg2DccbR.parquet has 0 missing values
7LxbgPH3grqf6lCpLKEwo7.parquet: (828, 5514)
7LxbgPH3grqf6lCpLKEwo7.parquet has 0 missing values
7r367wUYs1EvyBbeyOcq39.parquet: (921, 5514)
7r367wUYs1EvyBbeyOcq39.parquet has 0 missing values
6hoNS9LR1Lxb4EzMX72kXR.parquet: (5901, 5514)
6hoNS9LR1Lxb4EzMX72kXR.parquet has 5513 missing values
7vxD3WNDRkigLnIDHyy0cu.parquet: (930, 5514)
7vxD3WNDRkigLnIDHyy0cu.parquet has 0 missing values
2DCEgkjSeRQyMt8KovG8vF.parquet: (838, 5514)
2DCEgkjSeRQyMt8KovG8vF.parquet has 0 missing values
2hgBlERSFYDWndqjWNOV6v.parquet: (4765, 5514)
2hgBlERSFYDWndqjWNOV6v.parquet has 44104 missing values
0pIwpmg5oPcMWJXVSyrx4E.parqu

### 9 rows for podcasts have missing values (after checking, appears to be just beginning and end of podcast with silence)

In [21]:
print("printing column name where null is present in 6hoNS9LR1Lxb4EzMX72kXR.parquet:")
col_name = loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet'].columns.to_series()[np.isnan(loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet']).any()]
print(len(col_name))

print('Number rows where null is present:')
print(loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet'].shape[0] - loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet'].dropna().shape[0])
print('Row(s) with missing values:')
print(loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet'][loaded_parquet_df['6hoNS9LR1Lxb4EzMX72kXR.parquet'].isnull().any(axis=1)])

printing column name where null is present in 6hoNS9LR1Lxb4EzMX72kXR.parquet:
5513
Number rows where null is present:
1
Row(s) with missing values:
    0   1   2   3   4   5   6   7   8   9  ...  5504  5505  5506  5507  5508  \
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   NaN   

   5509  5510  5511  5512  y  
0   NaN   NaN   NaN   NaN  0  

[1 rows x 5514 columns]


In [22]:
print("printing column name where null is present in 2hgBlERSFYDWndqjWNOV6v.parquet:")
col_name = loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet'].columns.to_series()[np.isnan(loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet']).any()]
print(len(col_name))

print('Number rows where null is present:')
print(loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet'].shape[0] - loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet'].dropna().shape[0])
print('Row(s) with missing values:')
print(loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet'][loaded_parquet_df['2hgBlERSFYDWndqjWNOV6v.parquet'].isnull().any(axis=1)])

printing column name where null is present in 2hgBlERSFYDWndqjWNOV6v.parquet:
5513
Number rows where null is present:
8
Row(s) with missing values:
       0   1   2   3   4   5   6   7   8   9  ...  5504  5505  5506  5507  \
4757 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4758 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4759 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4760 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4761 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4762 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4763 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   
4764 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...   NaN   NaN   NaN   NaN   

      5508  5509  5510  5511  5512  y  
4757   NaN   NaN   NaN   NaN   NaN  0  
4758   NaN   NaN   NaN   NaN   NaN  0  
4759   NaN   NaN   NaN   NaN   NaN  0 

## Creating train-test sets

In [9]:
logreg = LogisticRegression(penalty = 'l2', class_weight ='balanced', max_iter = 1e10)

logreg_model_metrics = iid_modeling_loocv(loaded_parquet_df, logreg)

1jfWVd1sTapjjkQwDM4yQX.parquet will be the test podcast
Training set size: (29410, 5514)
Test set size: (714, 5514)
1jfWVd1sTapjjkQwDM4yQX.parquet scores: accuracy = 0.8431, precision = 0.0000, recall = 0.0000, f1_score = 0.0000
TN=602   FP=109   FN=3   TP=0
1jfWVd1sTapjjkQwDM4yQX.parquet completed in 37.65 seconds

0TkGYYIPwRqx8xzP0XGvRG.parquet will be the test podcast
Training set size: (27110, 5514)
Test set size: (3014, 5514)
0TkGYYIPwRqx8xzP0XGvRG.parquet scores: accuracy = 0.8945, precision = 0.0372, recall = 0.0957, f1_score = 0.0536
TN=2687   FP=233   FN=85   TP=9
0TkGYYIPwRqx8xzP0XGvRG.parquet completed in 31.66 seconds

79aiOALBohH9vEIUHPAoRj.parquet will be the test podcast
Training set size: (24521, 5514)
Test set size: (5603, 5514)
79aiOALBohH9vEIUHPAoRj.parquet scores: accuracy = 0.9099, precision = 0.0221, recall = 0.1389, f1_score = 0.0381
TN=5088   FP=443   FN=62   TP=10
79aiOALBohH9vEIUHPAoRj.parquet completed in 30.00 seconds

1XgTQnRlfJ0zpDdg2DccbR.parquet will be 

In [10]:
logreg_model_metrics

{'1jfWVd1sTapjjkQwDM4yQX.parquet': {'model': LogisticRegression(class_weight='balanced', max_iter=10000000000.0),
  'y_pred': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## Create a single model used for testing Random Forest and Adaboost

In [23]:
train_dfs = []

test_podcast = '7LxbgPH3grqf6lCpLKEwo7.parquet'
for key, val in loaded_parquet_df.items():
    if key != test_podcast:
        train_dfs.append(val)
    
# get the training set
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
train_set = pd.concat(train_dfs).dropna()
print(train_set.shape)

# get the test set
test_set = loaded_parquet_df[test_podcast].dropna()
test_set.shape

(29296, 5514)


(828, 5514)

In [24]:
X_train, y_train = train_set.drop('y', axis=1), train_set['y']
print(X_train.shape)
print(y_train.shape)

(29296, 5513)
(29296,)


In [25]:
X_test, y_test = test_set.drop('y', axis=1), test_set['y']
print(X_test.shape)
print(y_test.shape)

(828, 5513)
(828,)


# Modeling with Random Forest

In [45]:
X_train_smaller = X_train.head(10000)
y_train_smaller = y_train.head(10000)

In [46]:
%%time
rmf = RandomForestClassifier(random_state = 34)
rmf.fit(X_train_smaller, y_train_smaller)

y_pred_rmf = rmf.predict(X_test)
y_pred_prob_rmf = rmf.predict_proba(X_test)
y_pred_prob_log_rmf = rmf.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(rmf.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.99
CPU times: user 7min 47s, sys: 22.4 ms, total: 7min 47s
Wall time: 7min 47s


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:722: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [47]:
print(classification_report(y_test, y_pred_rmf))
print(confusion_matrix(y_test, y_pred_rmf))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       816
           1       0.00      0.00      0.00        12

    accuracy                           0.99       828
   macro avg       0.49      0.50      0.50       828
weighted avg       0.97      0.99      0.98       828

[[816   0]
 [ 12   0]]


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [48]:
X_train_smaller = X_train.head(12000)
y_train_smaller = y_train.head(12000)

In [ ]:
%%time
rmf = RandomForestClassifier(random_state = 34)
rmf.fit(X_train_smaller, y_train_smaller)

y_pred_rmf = rmf.predict(X_test)
y_pred_prob_rmf = rmf.predict_proba(X_test)
y_pred_prob_log_rmf = rmf.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(rmf.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.99
CPU times: user 12min 29s, sys: 42.8 ms, total: 12min 29s
Wall time: 12min 29s


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:722: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [ ]:
print(classification_report(y_test, y_pred_rmf))
print(confusion_matrix(y_test, y_pred_rmf))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       816
           1       0.00      0.00      0.00        12

    accuracy                           0.99       828
   macro avg       0.49      0.50      0.50       828
weighted avg       0.97      0.99      0.98       828

[[816   0]
 [ 12   0]]


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [ ]:
X_train_smaller = X_train.head(15000)
y_train_smaller = y_train.head(15000)

In [ ]:
%%time
rmf = RandomForestClassifier(random_state = 34)
rmf.fit(X_train_smaller, y_train_smaller)

y_pred_rmf = rmf.predict(X_test)
y_pred_prob_rmf = rmf.predict_proba(X_test)
y_pred_prob_log_rmf = rmf.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(rmf.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.99
CPU times: user 20min, sys: 33.8 ms, total: 20min
Wall time: 20min


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/ensemble/_forest.py:722: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


In [ ]:
print(classification_report(y_test, y_pred_rmf))
print(confusion_matrix(y_test, y_pred_rmf))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       816
           1       0.00      0.00      0.00        12

    accuracy                           0.99       828
   macro avg       0.49      0.50      0.50       828
weighted avg       0.97      0.99      0.98       828

[[816   0]
 [ 12   0]]


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

## Modeling with AdaBoost

In [60]:
%%time

# Initialise the model with Decision Tree classifier as the base model
boost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(), 
                            algorithm = 'SAMME.R', learning_rate=1,random_state = 10)

X_train_smaller = X_train.head(1000)
y_train_smaller = y_train.head(1000)

# Fit on the entire data
boost.fit(X_train_smaller,y_train_smaller)

y_pred_boost = boost.predict(X_test)
y_pred_prob_boost = boost.predict_proba(X_test)
y_pred_prob_log_boost = boost.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(boost.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.98
CPU times: user 3.5 s, sys: 6 µs, total: 3.5 s
Wall time: 3.55 s


In [61]:
print(confusion_matrix(y_test, y_pred_boost))
print(classification_report(y_test, y_pred_boost))

[[812   4]
 [ 12   0]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       816
           1       0.00      0.00      0.00        12

    accuracy                           0.98       828
   macro avg       0.49      0.50      0.50       828
weighted avg       0.97      0.98      0.98       828



In [62]:
%%time

# Initialise the model with Decision Tree classifier as the base model
boost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(), 
                            algorithm = 'SAMME.R', learning_rate=1,random_state = 10)

X_train_smaller = X_train.head(2000)
y_train_smaller = y_train.head(2000)

# Fit on the entire data
boost.fit(X_train_smaller,y_train_smaller)

y_pred_boost = boost.predict(X_test)
y_pred_prob_boost = boost.predict_proba(X_test)
y_pred_prob_log_boost = boost.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(boost.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.97
CPU times: user 55.2 s, sys: 339 µs, total: 55.2 s
Wall time: 55.2 s


In [63]:
print(confusion_matrix(y_test, y_pred_boost))
print(classification_report(y_test, y_pred_boost))

[[801  15]
 [ 12   0]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       816
           1       0.00      0.00      0.00        12

    accuracy                           0.97       828
   macro avg       0.49      0.49      0.49       828
weighted avg       0.97      0.97      0.97       828



In [64]:
%%time

# Initialise the model with Decision Tree classifier as the base model
boost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(), 
                            algorithm = 'SAMME.R', learning_rate=1,random_state = 10)

X_train_smaller = X_train.head(4000)
y_train_smaller = y_train.head(4000)

# Fit on the entire data
boost.fit(X_train_smaller,y_train_smaller)

y_pred_boost = boost.predict(X_test)
y_pred_prob_boost = boost.predict_proba(X_test)
y_pred_prob_log_boost = boost.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(boost.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.97
CPU times: user 3min 8s, sys: 8.97 ms, total: 3min 8s
Wall time: 3min 8s


In [65]:
print(confusion_matrix(y_test, y_pred_boost))
print(classification_report(y_test, y_pred_boost))

[[807   9]
 [ 12   0]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       816
           1       0.00      0.00      0.00        12

    accuracy                           0.97       828
   macro avg       0.49      0.49      0.49       828
weighted avg       0.97      0.97      0.97       828



In [66]:
%%time

# Initialise the model with Decision Tree classifier as the base model
boost = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(), 
                            algorithm = 'SAMME.R', learning_rate=1,random_state = 10)

X_train_smaller = X_train.head(15000)
y_train_smaller = y_train.head(15000)

# Fit on the entire data
boost.fit(X_train_smaller,y_train_smaller)

y_pred_boost = boost.predict(X_test)
y_pred_prob_boost = boost.predict_proba(X_test)
y_pred_prob_log_boost = boost.predict_log_proba(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(boost.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.96
CPU times: user 55min 32s, sys: 86.3 ms, total: 55min 32s
Wall time: 55min 32s


In [67]:
print(confusion_matrix(y_test, y_pred_boost))
print(classification_report(y_test, y_pred_boost))

[[798  18]
 [ 12   0]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       816
           1       0.00      0.00      0.00        12

    accuracy                           0.96       828
   macro avg       0.49      0.49      0.49       828
weighted avg       0.97      0.96      0.97       828

